In [1]:
import itertools

from trec_car.read_data import iter_annotations

from tools.dataloaders import WikipediaCBOR, PageFormat

import pandas as pd

"""
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.column import Column
from pyspark.sql.types import StructType, ArrayType, StringType, DataType

from pyspark.sql import SparkSession, SQLContext


spark = SparkSession.builder.appName("colla-tokenizer").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
"""

'\nfrom pyspark.sql.dataframe import DataFrame\nfrom pyspark.sql.column import Column\nfrom pyspark.sql.types import StructType, ArrayType, StringType, DataType\n\nfrom pyspark.sql import SparkSession, SQLContext\n\n\nspark = SparkSession.builder.appName("colla-tokenizer").getOrCreate()\nsc = spark.sparkContext\nsqlContext = SQLContext(sc)\n'

In [2]:
wikipedia_cbor = WikipediaCBOR("wikipedia/car-wiki2020-01-01/enwiki2020.cbor",
                                "wikipedia/car-wiki2020-01-01/partitions",
                                # repreprocess=True, page_lim=1000
                                )

Loaded from cache


In [11]:
from typing import cast, Any, Optional, Tuple, Union, NamedTuple

from trec_car.read_data import (AnnotationsFile, Page, Para,
                                Section, List as ParaList, ParaLink, ParaText, ParaBody)

from tokenizers.pre_tokenizers import Whitespace
from collections import defaultdict

from io import StringIO

print("Test")

def preprocess_page_new(enumerated_page: Tuple[int, Page]):
    """
    Transform a list of pages into a flattened representation that can
    then be easily (de)serialized.
    """

    id, page = enumerated_page

    # For the sake of easy link spans they are byte oriented to make
    # it easier for the rust std
    # page_content = StringIO()
    split_content = []
    orig_page_content = StringIO()
    links = []

    splitter = Whitespace()
    # splitter = 0

    # Encode a link. Cast to padding if the link was not "common".
    # Call this method only after preprocessing has been done!
    def encode_link(link):
        #return self.key_encoder.get(link, 0)
        return wikipedia_cbor.key_encoder.get(link, 0)

    # People say pattern matching is overrated.
    # I beg to differ.
    # (It's also true that a tree structure for tokenization makes
    # absolutely no sense - but I don't get to decide things apparently).
    def handle_section(skel: Section):
        for subskel in skel.children:
            visit_section(subskel)

    def handle_list(skel: ParaList):
        visit_section(skel.body)

    def handle_para(skel: Para):
        paragraph = skel.paragraph
        bodies = paragraph.bodies

        for body in bodies:
            visit_section(body)

    def handle_paratext(body: ParaBody):
        split_body = splitter.pre_tokenize_str(body.get_text())
        #print('from handle_paratext: "' + body.get_text() + '"')

        # take care of the space...
        running_prefix = 0
        if len(split_content) > 0:
            running_prefix = split_content[-1][1][1]
        
        while running_prefix < len(orig_page_content.getvalue()) and orig_page_content.getvalue()[running_prefix] == " ":
            running_prefix += 1

        orig_page_content.write(body.get_text())
        
        #if len(split_content) > 0:
        #    running_prefix -= split_content[0][1][0]


        #print(f"After skipping: {orig_page_content.getvalue()[running_prefix:]}")

        split_body = [(text, (begin_offset + running_prefix,
                                end_offset + running_prefix)) for text, (begin_offset, end_offset) in split_body]

        # print(split_body)

        for tok, (begin, end) in split_body:
            assert tok == orig_page_content.getvalue()[begin:end], f"generated {orig_page_content.getvalue()[begin:end]} but expected {tok}"
        split_content.extend(split_body)

    def handle_paralink(body: ParaLink):
        encoded_link = encode_link(body.page)
        split_body = splitter.pre_tokenize_str(body.get_text())

        #print('from handle_paralink: "' + body.get_text() + '"')

        #running_prefix = running_length_count# + int(running_length_count != 0)
        if len(split_content) > 0:
            running_prefix = split_content[-1][1][1]
        
        while running_prefix < len(orig_page_content.getvalue()) and orig_page_content.getvalue()[running_prefix] == " ":
            running_prefix += 1

        orig_page_content.write(body.get_text())
        
        
        if len(split_content) > 0:
            running_prefix -= split_content[0][1][0]


        split_body = [(text, (begin_offset + running_prefix,
                                end_offset + running_prefix)) for text, (begin_offset, end_offset) in split_body]

        split_content.extend(split_body)

        if len(split_body) > 0:
            end_byte_span = split_body[-1][1][1] - 1
            start_mention_idx = len(split_content) - len(split_body)
            links.append((encoded_link, start_mention_idx, len(split_content)))

        for tok, (begin, end) in split_body:
            assert tok == orig_page_content.getvalue()[begin:end], f"generated {orig_page_content.getvalue()[begin:end]} but expected {tok}"


    def nothing():
        return lambda body: None

    handler = defaultdict(nothing, {Section: handle_section,
                                    Para: handle_para,
                                    ParaList: handle_list,
                                    ParaLink: handle_paralink,
                                    ParaText: handle_paratext})

    def visit_section(skel):
        # Recur on the sections
        handler[type(skel)](skel)

    for skel in page.skeleton:
        visit_section(skel)

    return id, page.page_name, orig_page_content.getvalue(), split_content, links

    # return PageFormat(id, page.page_name, page_content.getvalue(), links)

Test


In [12]:

# a = wikipedia_cbor[0]
k = 5
with open(wikipedia_cbor.cbor_path, "rb") as cbor_file:
    topk = itertools.islice(iter_annotations(cbor_file), k)
    topk = map(preprocess_page_new, enumerate(topk))

    df = pd.DataFrame(topk, columns=["id", "title", "text", "tokenized_text", "links"])
    #df = sqlContext.createDataFrame(topk)
    # page = next(iter_annotations(cbor_file))
    # dataloader_page = wikipedia_cbor.preprocess_page(page)
    # spacy_collect_mentions(dataloader_page)


In [13]:
df.head()

#df[2][0][:20]

,id,title,text,tokenized_text,links
0,0,Anarchism,Anarchism is an anti-authoritarian political a...,"[(Anarchism, (0, 9)), (is, (10, 12)), (an, (13...","[(3818014, 3, 6), (4210689, 6, 7), (5408297, 8..."
1,1,Albedo,"Albedo () (, meaning 'whiteness') is the measu...","[(Albedo, (0, 6)), ((), (7, 9)), ((,, (10, 12)...","[(3849449, 12, 14), (6970944, 15, 17), (612884..."
2,2,Autism,Autism is a developmental disorder characteriz...,"[(Autism, (0, 6)), (is, (7, 9)), (a, (10, 11))...","[(5748136, 3, 5), (215297, 49, 50), (5759583, ..."
3,3,A,A or a is the first letter and the first vowel...,"[(A, (0, 1)), (or, (2, 4)), (a, (5, 6)), (is, ...","[(4796107, 6, 7), (4772344, 10, 12), (4357688,..."
4,4,Achilles,"In Greek mythology, Achilles ( ) or Achilleus ...","[(In, (0, 2)), (Greek, (3, 8)), (mythology, (9...","[(4207422, 1, 3), (5079607, 16, 18), (6013052,..."


In [14]:
import spacy

nlp = spacy.load("xx_ent_wiki_sm")

In [15]:
from tools.dataloaders import TokenizedText
from trie_search import RecordTrieSearch

class MyRecordTrie(RecordTrieSearch):
    def __init__(self, records):
        super().__init__("<Q", records)

    def search_all_patterns(self, tokens: TokenizedText):
        words = [tok[0] for tok in tokens]

        for i, (word, span) in enumerate(tokens):
            for pattern in self._TrieSearch__search_prefix_patterns(word, words[i+1:]):
                weight = trie[pattern][0][0]
                yield pattern, span[0], weight, i # exact token position
    
    def search_longest_patterns(self, tokens):
        # avoid overlapping mentions
        all_patterns = self.search_all_patterns(tokens)
        check_field = [0] * len(tokens)
        for pattern, start_idx, weight, idx in sorted(
                all_patterns, key=lambda x: len(x[0]), reverse=True):
            pattern_length = pattern.count(" ") + 1
            target_field = check_field[idx:idx + pattern_length]
            check_sum = sum(target_field)
            if check_sum == 0:
                for i in range(pattern_length):
                    check_field[idx + i] = 1
                yield pattern, start_idx, weight, idx

In [16]:
from typing import List, Tuple
#from pyspark.sql.functions import pandas_udf
# from spacy.attrs import LOWER, ENT_TYPE
import numpy as np
import pandas as pd

from trie_search import RecordTrieSearch
from heapq import merge

def autolink(
    page_id: int,
    title: str,
    text: str,
    tokenized_text: List[Tuple[str, Tuple[int, int]]],
    links: List[Tuple[int, int, int]]
) -> List[Tuple[int, int, int]]:
    # title_toks = nlp(title)
    title_toks = Whitespace().pre_tokenize_str(title)

    link_idx = 0

    exact_mentions = {}
    mention_builder = []

    # TODO: deal with ambiguities...
    exact_mentions[" ".join(str(tok[0]) for tok in title_toks)] = page_id

    #print(exact_mentions)

    remapped_links = []
    for link in links:
        # revert the tokenization algorithm
        start_byte = tokenized_text[link[1]][1][0]
        end_byte = tokenized_text[link[2]-1][1][1]

        #print(text[start_byte:end_byte])

        exact_mentions[text[start_byte:end_byte]] = link[0]
        remapped_links.append((link[0], start_byte, end_byte))
    

    #print(list(map(lambda x: (x[0], (x[1],)), exact_mentions.items())))

    trie = MyRecordTrie(map(lambda x: (x[0], (x[1],)), exact_mentions.items()))

    #print(trie.keys())

    # print(trie.items())
    patterns = sorted(trie.search_longest_patterns(tokenized_text), key=lambda x: x[1]) # sort by apparition


    link = None
    link_idx = 0
    new_links = []

    for title, new_link_id, idx, token_position in patterns:
        new_link = (title, new_link_id, idx, idx + len(title))
        #print(new_link)

        if link_idx < len(links):
            link = links[link_idx]
        
        while link_idx < len(links) - 1 and idx > link[2]:
            link_idx += 1
            link = links[link_idx]

        if link_idx >= len(links):
            link = None
        
        if link is None or idx < link[1]:
            new_links.append(new_link)
    
    # return exact_mentions

    return list(merge(remapped_links, new_links, key=lambda x: x[1]))

In [17]:
page_id = 0
title = "Anarchism"
tokenized_text = df.iloc[0]["tokenized_text"]

text = df.iloc[0]["text"]
links = df.iloc[0]["links"]

autolink(page_id, title, text, tokenized_text, links)

In [1]:
# autolink(0, "Anarchy", *list(df.iloc[0])[2:])

In [10]:
from trie_search import RecordTrieSearch

patterns = [("Albert Einstein", (1,)), ("Einstein", (1,)), ("Albert Hitchcock", (2,))]
t = RecordTrieSearch("<Q", patterns)

[(1,)]

In [87]:

splitter = Whitespace()



for p in search_all_patterns("This is a a little lesson in trickery, this is going down in history!."):
    print(p)

['This', 'is', 'a', 'a', 'little', 'lesson', 'in', 'trickery', ',', 'this', 'is', 'going', 'down', 'in', 'history', '!.']
('trickery', 29, 3)
('history', 61, 2)


In [54]:
trie.splitter

'\\w+|[^\\w\\s]+'